# Overview

## Setup

In [ ]:
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_chroma import Chroma

llm = ChatVertexAI(model="gemini-1.0-pro")
embeddings = VertexAIEmbeddings(model="text-embedding-004")
vector_store = Chroma(embedding_function=embeddings)

In [ ]:
import bs4
import uuid
from typing import Annotated, Sequence
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, BaseMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import START, END, StateGraph, MessagesState
from typing_extensions import List, TypedDict
from IPython.display import Image, display

## QA RAG model
see: https://python.langchain.com/docs/tutorials/rag/

### Pipeline for ingesting data from a source and indexing it (by semantic search for our case)

1. Load data with a data loader
2. Break large documents into smaller chunks with text splitters to fit into model's finite context window
3. Use vector store and embeddings model to store and index the splits

In [ ]:
# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",
               "https://docs.thena.fi/thena?ref=bnbchain.ghost.io",
               "https://docs.thena.fi/thena/the-onboarding",
               "https://docs.thena.fi/thena/the-spot-dex/swap-guide",
               "https://docs.thena.fi/thena/the-spot-dex/limit-order",
               "https://docs.thena.fi/thena/the-liquidity-pools/introduction-to-fusion",
               "https://docs.thena.fi/thena/the-liquidity-pools/liquidity-pools-typology",
               "https://docs.thena.fi/thena/the-liquidity-pools/earn-the",
               "https://docs.thena.fi/thena/the-liquidity-pools/earn-trading-fees",
                ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer( 
            # Only keep post title, headers, and content from the full HTML.
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

### Retrieval and generation chain for taking user query at run time and retrieving data from index

1. Use a retriever to match against user query. Extending this to a tool allows models to rewrite user queries into more effective search queries. This also gives model a choice to either respond immediately or do RAG.
2. ChatModel / LLM produces answer from prompt (which takes in user query and retrieved data)

In [ ]:

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(
        query, 
        k=2
    )
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])

## Agent
Now the control flow is defined by the reasoning capabilities of LLMs. Using agents allows you to offload additional discretion over the retrieval process. Although their behavior is less predictable than the above "chain", they are able to execute multiple retrieval steps in service of a query, or iterate on a single search.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver() # support multiple conversational turns

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

websearch = TavilySearchResults(max_results=2)

mem_agent = create_react_agent(llm, [retrieve, websearch], checkpointer=memory)
# display(Image(agent_executor.get_graph().draw_mermaid_png()))

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate, SystemMessagePromptTemplate

def create_tool_agent(llm: ChatVertexAI, tools: list, system_prompt: str):
    """Helper function to create agents with custom tools and system prompt
    Args:
        llm (ChatVertexAI): LLM for the agent
        tools (list): list of tools the agent will use
        system_prompt (str): text describing specific agent purpose

    Returns:
        executor (AgentExecutor): Runnable for the agent created.
    """
    
    # Each worker node will be given a name and some tools.
    
    system_prompt_template = PromptTemplate(

                template= system_prompt + """
                ONLY respond to the part of query relevant to your purpose.
                IGNORE tasks you can't complete. 
                Use the following context to answer your query 
                if available: \n {agent_history} \n
                """,
                input_variables=["agent_history"],
            )

    #define system message
    system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt_template)

    prompt = ChatPromptTemplate.from_messages(
        [system_message_prompt,
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_tool_calling_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools, 
                return_intermediate_steps= True, verbose = False)
    return executor

In [ ]:
sentiment_prompt = """ You are a assistant for performing research on market sentiment of a cryto asset.
        You are to gauge whether the sentiment is positive, negative or neutral based on news and statements
        made by influential sources in the cryptocurrency space. Also, weigh the sentiment based on the profile of the source.
        For example, a statement from a well-known cryptocurrency influencer may have more weight than a random twitter user with less than average number of followers.
        You must also take into account of whether the opinion is based on facts or speculation, and if the source has a history of being accurate.
        Use your tools to answer questions. If you do not have a tool to
        answer the question, say so. """

sentiment_agent = create_tool_agent(llm=llm, tools = [retrieve, websearch], # todo: add twitter, governance, news tools
              system_prompt = sentiment_prompt)

risk_prompt = """ You are a assistant for performing risk analysis on a crypto asset.
        You are to assess the risk of investing in a cryptocurrency based on the information available.
        You must consider the technology behind the cryptocurrency, the team behind the project, the market conditions, and the regulatory environment.
        Use your tools to complete requests. If you do not have a tool to
       complete the request, say so. """

risk_agent = create_tool_agent(llm=llm, tools = [retrieve, websearch], 
                    system_prompt = risk_prompt)


thena_api_prompt = """ You are an assistant for performing action such as querying user wallet status, trading, swapping and liquidity provision to THENA blockchain ecosystem based on the user's request if any.
        Use your tools to complete requests. If you do not have a tool to
       complete the request, say so. """

thena_api_agent = create_tool_agent(llm=llm, tools = [], 
                    system_prompt = thena_api_prompt)


xrpl_api_prompt = """ You are an assistant for performing action such as querying user wallet status, trading, swapping and liquidity provision to Ripple Ledger (XRPL) blockchain ecosystem based on the user's request if any.
        Use your tools to complete requests. If you do not have a tool to
       complete the request, say so. """

xrpl_api_agent = create_tool_agent(llm=llm, tools = [], # tools for staking, LPing, swapping etc
                    system_prompt = xrpl_api_prompt)

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

system_prompt_template = PromptTemplate(

      template= """ You are a helpful assistant that summarises agent history 
                      in response to the original user query below. 
                      SUMMARISE ALL THE OUTPUTS AND TOOLS USED in agent_history.
                      The agent history is as follows: 
                        \n{agent_history}\n""",
                input_variables=["agent_history"],  )

system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt_template)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_prompt,
        MessagesPlaceholder(variable_name="messages"),
    ])

comms_agent = (prompt| llm) 

In [ ]:
for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content="Can you do an interactive analysis for what cryptocurrency I should buy?"
                )
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import JsonOutputParser
from enum import Enum
members = ["Sentiment", "Risk", "THENA_API", "XRPL_API", "Mem", "Communicate"]

#create options map for the supervisor output parser.
member_options = {member:member for member in members}

#create Enum object
MemberEnum = Enum('MemberEnum', member_options)

from pydantic import BaseModel

#force Supervisor to pick from options defined above
# return a dictionary specifying the next agent to call 
#under key next.
class SupervisorOutput(BaseModel):
    #defaults to communication agent
    next: MemberEnum = MemberEnum.Communicate


system_prompt = (
    """You are a supervisor tasked with managing a conversation between the
    crew of workers:  {members}. Given the following user request, 
    and crew responses respond with the worker to act next.
    Each worker will perform a task and respond with their results and status. 
    When finished with the task, route to communicate to deliver the result to 
    user. Given the conversation and crew history below, who should act next?
    Hint: API agents should take into account of sentiment and risk analysis.
    Analysis should take into account mem agent history. 
    Treat mem agent like representative of user and their portfolio.
    Select one of: {options} 
    \n{format_instructions}\n"""
)
# Our team supervisor is an LLM node. It just picks the next agent to process
# and decides when the work is completed

# Using openai function calling can make output parsing easier for us
supervisor_parser = JsonOutputParser(pydantic_object=SupervisorOutput)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_history")
       
    ]
).partial(options=str(members), members=", ".join(members), 
    format_instructions = supervisor_parser.get_format_instructions())


supervisor_chain = (
    prompt | llm |supervisor_parser
)

## Graph

In [ ]:

from langchain_core.messages import AIMessage
import operator

# For agents in the crew 
def crew_nodes(state, crew_member, name):
    #read the last message in the message history.
    input = {'messages': [state['messages'][-1]], 
                'agent_history' : state['agent_history']}
    result = crew_member.invoke(input)
    #add response to the agent history.
    return {"agent_history": [AIMessage(content= result["output"], 
              additional_kwargs= {'intermediate_steps' : result['intermediate_steps']}, 
              name=name)]}

def comms_node(state):
    #read the last message in the message history.
    input = {'messages': [state['messages'][-1]],
                     'agent_history' : state['agent_history']}
    result = comms_agent.invoke(input)
    #respond back to the user.
    return {"messages": [result]}

# The agent state is the input to each node in the graph
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str 
    agent_history: Annotated[Sequence[BaseMessage], operator.add]

In [ ]:
from functools import partial

workflow = StateGraph(AgentState)

mem_node = partial(crew_nodes, crew_member=mem_agent, name="Memory")
sentiment_node = partial(crew_nodes, crew_member=sentiment_agent, name="Sentiment")
risk_node = partial(crew_nodes, crew_member=risk_agent, name="Risk")
thena_api_node = partial(crew_nodes, crew_member=thena_api_agent, name="THENA_API")
xrpl_api_node = partial(crew_nodes, crew_member=xrpl_api_agent, name="XRPL_API")

workflow.add_node("Mem", mem_node)
workflow.add_node("Sentiment", sentiment_node)
workflow.add_node("Risk", risk_node)
workflow.add_node("THENA_API", thena_api_node)
workflow.add_node("XRPL_API", xrpl_api_node)
workflow.add_node("Communicate", comms_node )
workflow.add_node("Supervisor", supervisor_chain)
workflow.set_entry_point("Supervisor")
workflow.add_edge('Mem', "Supervisor")
workflow.add_edge('Sentiment', "Supervisor")
workflow.add_edge('Risk', "Supervisor")
workflow.add_edge('THENA_API', "Supervisor")
workflow.add_edge('XRPL_API', "Supervisor")
workflow.add_edge('Communicate', END) 
# end loop at communication agent.

# The supervisor populates the "next" field in the graph state
# which routes to a node or finishes
workflow.add_conditional_edges("Supervisor", lambda x: x["next"], member_options)

graph = workflow.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

## Tests / Examples

In [ ]:
from langchain_core.messages import HumanMessage

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Can you perform an interactive analysis for what cryptocurrency I should buy?")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")